<a href="https://colab.research.google.com/github/mc-friday/hanghaeAI/blob/main/%5B5%EC%A3%BC%EC%B0%A8%5D%EA%B8%B0%EB%B3%B8%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [170]:
!pip install langchain-community langchain-chroma langchain-openai bs4 openai

In [171]:
import os
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from collections import Counter
from langchain.schema import Document
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

## 1. 환경 변수에 저장된 OpenAI API 키 가져오기

In [172]:
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

## 2. LangChain의 GPT 모델 설정 (GPT-4o-mini 사용)

In [173]:
# ChatGPT 기반 LLM 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini",  # ChatGPT 기반 모델
    api_key=openai_api_key,
    temperature=0
)


## 3. [MY_DATA] 함수 정의

In [174]:
# 3.1. 데이터 로드 및 처리
def fetch_web_data(url):
    """
    웹 페이지에서 데이터를 가져오고 BeautifulSoup으로 파싱.
    """
    response = requests.get(url)
    response.encoding = 'utf-8'  # UTF-8 인코딩
    soup = BeautifulSoup(response.text, "html.parser")
    content = soup.find_all(class_="editedContent")

    if not content:
        print("No content found on the page.")
        return []

    docs = [Document(page_content=c.get_text(strip=True)) for c in content]
    return docs

# 3.2. 문서 분할
def split_documents(docs, chunk_size=3000, overlap=600):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap
    )
    splits = splitter.split_documents(docs)
    print(f"Number of splits created: {len(splits)}")
    return splits

# 3.3. 사용자 질문 및 검색
def generate_summary(query, retriever, llm):
    """
    사용자의 질문에 따라 문서를 검색하고 요약을 생성.
    """
    # Retriever로 관련 문서 검색
    retrieved_docs = retriever.invoke(query)
    context = "\n\n".join(doc.page_content for doc in retrieved_docs)

    # 프롬프트 템플릿 정의
    prompt_template = PromptTemplate(
        input_variables=["context", "question"],
        template="""
        다음은 관련 문서 내용입니다:

        {context}

        질문: {question}

        위 문서를 바탕으로 질문에 대해 명확하고 간결하게 답변하세요.
        """
    )

    # 프롬프트 생성 및 ChatOpenAI 호출
    prompt = prompt_template.format(context=context, question=query)
    response = llm.invoke([{"role": "user", "content": prompt}])

    # AIMessage 객체에서 content 추출
    return response.content

## 4. 실행 및 결과 출력

In [175]:
url = "https://spartacodingclub.kr/blog/all-in-challenge_winner"
documents = fetch_web_data(url)

if not documents:
    raise ValueError("Failed to load documents. Please check the URL or content structure.")

print(f"Loaded {len(documents)} documents.")


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(documents)

# OpenAI 임베딩 생성
embedding = OpenAIEmbeddings(api_key=openai_api_key)

# Chroma 벡터스토어 생성
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embedding
)

retriever = vectorstore.as_retriever()

query = "ALL-in 코딩 공모전 수상작들을 요약하고 어떤 상을 수상했는지 정리해줘."
response = generate_summary(query, retriever, llm)

# 결과 출력
print("\n=== 질문 ===")
print(query)
print("\n=== 응답 ===")
print(response)

Loaded 1 documents.

=== 질문 ===
ALL-in 코딩 공모전 수상작들을 요약하고 어떤 상을 수상했는지 정리해줘.

=== 응답 ===
ALL-in 코딩 공모전에서 수상한 작품들은 다음과 같습니다:

1. **대상**: [Lexi Note] 언어공부 필기 웹 서비스
   - **제작자**: 다나와(김다애, 박나경)
   - **내용**: 어문학 전공 학생들이 효율적으로 언어를 학습할 수 있도록 돕는 웹 서비스. 단어를 드래그하면 네이버 사전과 연동되어 의미를 찾고, 번역 버튼을 통해 긴 문장을 쉽게 이해할 수 있으며, 할일 목록과 스케줄 템플릿을 제공하여 학습 효율을 높임.

이 외에도 다른 수상작들이 있을 수 있으나, 문서에서는 Lexi Note에 대한 정보만 제공되고 있습니다.
